<a href="https://colab.research.google.com/github/keshavchandel13/DAA-lab/blob/main/p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def calc(a,b):
  yield print("The sum is: ",a+b)
  yield print("The sub is: ",a-b)
  yield print("The multi is: ",a*b)
  yield print("The divide is: ",a/b)
x =  calc(4,2)
for i in x:
  pass

The sum is:  6
The sub is:  2
The multi is:  8
The divide is:  2.0


In [ ]:
def gen(lst):
  for i in lst:
    yield i
lst = [1,2,3]
sq = [i**2 for i in lst]
res = gen(sq)
for i in res:
  print(i)

1
4
9


In [1]:
pip install simpy

# write a program in simpy to simulate the realtime process using a car, where the car is alternatively driving and parking for the specified duration of time

In [ ]:
def Car(env):
  while(True):
    print("The car is at parking, Time is: ",env.now)
    parking_time = 3 # Parking time is fixed here
    yield env.timeout(parking_time) # The car will be out of the parking after the fixed time

    print("The car is driving, Time is: ", env.now)
    driving_time = 5
    yield env.timeout(driving_time)

In [ ]:
import simpy
# Creating simpy environment
env = simpy.Environment()
env.process(Car(env))
env.run(until=25)

The car is at parking, Time is:  0
The car is driving, Time is:  3
The car is at parking, Time is:  8
The car is driving, Time is:  11
The car is at parking, Time is:  16
The car is driving, Time is:  19
The car is at parking, Time is:  24


# Process Interaction

In [ ]:
class Car:
  def __init__(self, env):
    self.env = env
    self.action = env.process(self.drive())
  def drive(self):
    while(True):
      # The car is parking
      print("The car is Parking and charging, Time: ",env.now)
      parking_time = 5 # Charging time for the car while its parking
      yield self.env.process(self.charge(parking_time))
      # The car is driving
      print("The car is driving, time: ",env.now)
      driving_time = 10
      yield env.timeout(driving_time)
  def charge(self, charging_time):
    yield self.env.timeout(charging_time)

In [ ]:
import simpy
env = simpy.Environment()
a = Car(env)
env.run(until=55)

The car is Parking, Time:  0
The car is driving, time:  5
The car is Parking, Time:  15
The car is driving, time:  20
The car is Parking, Time:  30
The car is driving, time:  35
The car is Parking, Time:  45
The car is driving, time:  50


# Process Interaction through interuption

In [ ]:
def driver_arrived(env, car):
  yield env.timeout(2)
  car.action.interrupt()
class Car:
  def __init__(self, env):
    self.env = env
    self.action = self.env.process(self.drive())
  def drive(self):
    while(True):
      print("The car is parking and charging, Time: ",env.now)
      parking_time = 7
      try:
        yield self.env.process(self.charge(parking_time))
      except simpy.Interrupt:
        print("Driver Sahab A chuke hai chalo")
      print("The car is driving, Time: ", env.now)
      driving_time = 5
      yield env.timeout(driving_time)
  def charge(self, charging_time):
    yield env.timeout(charging_time)

In [ ]:
import simpy
env = simpy.Environment()
car = Car(env)
env.process(driver_arrived(env,car))
env.run(until=55)

The car is parking and charging, Time:  0
Driver Sahab A chuke hai chalo
The car is driving, Time:  2
The car is parking and charging, Time:  7
The car is driving, Time:  14
The car is parking and charging, Time:  19
The car is driving, Time:  26
The car is parking and charging, Time:  31
The car is driving, Time:  38
The car is parking and charging, Time:  43
The car is driving, Time:  50


# TO implement the concept of shared resources

In [ ]:
def car(env,bcs, name,driving_time,parking_time):
  yield env.timeout(driving_time)
  # print(f"{name} is driving at time {env.now}")
  print(f"{name} The car reached at bcs at {env.now}")
  with bcs.request() as req:
    yield req
    print(f"{name} The car is parking and charging at {env.now}")
    yield env.timeout(parking_time)
    print(f"{name} The car leaved the bcs station at {env.now}")


In [ ]:
import simpy
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=3)

In [ ]:
for i in range(6):
  env.process(car(env,bcs,f'Car_{i}',i*2,10))

In [ ]:
env.run()

Car_0 The car reached at bcs at 0
Car_0 The car is parking and charging at 0
Car_1 The car reached at bcs at 2
Car_1 The car is parking and charging at 2
Car_2 The car reached at bcs at 4
Car_2 The car is parking and charging at 4
Car_3 The car reached at bcs at 6
Car_4 The car reached at bcs at 8
Car_5 The car reached at bcs at 10
Car_0 The car leaved the bcs station at 10
Car_3 The car is parking and charging at 10
Car_1 The car leaved the bcs station at 12
Car_4 The car is parking and charging at 12
Car_2 The car leaved the bcs station at 14
Car_5 The car is parking and charging at 14
Car_3 The car leaved the bcs station at 20
Car_4 The car leaved the bcs station at 22
Car_5 The car leaved the bcs station at 24


# To simulate Priorities in simpy

In [ ]:
def resource_user(name,env,resource,wait,prio):
  yield env.timeout(wait)
  with resource.request(priority=prio) as req:
    print(f"Car_{name} requested resource at {env.now}")
    yield req
    print(f"Car{name}got resource at {env.now}")
    yield env.timeout(4)


In [ ]:
import simpy
env = simpy.Environment()
res = simpy.PriorityResource(env, capacity=1)
p1 = env.process(resource_user(1, env, res, wait=4, prio=-5))
p2 = env.process(resource_user(2, env, res, wait=3, prio=-1))
p3 = env.process(resource_user(3, env, res, wait=2, prio=1))
p4 = env.process(resource_user(4, env, res, wait=1, prio=3))
env.run()

Car_4 requested resource at 1
Car4got resource at 1
Car_3 requested resource at 2
Car_2 requested resource at 3
Car_1 requested resource at 4
Car1got resource at 5
Car2got resource at 9
Car3got resource at 13


# question 1

In [17]:
def interrupt(env,car):
  yield env.timeout(2)
  car.action.interrupt()
class Car:
  def __init__(self,env,res,name):
    self.env = env
    self.res = res
    self.name = name
    self.action = env.process(self.drive())
  def drive(self):
    while(True):
      print(f"{self.name} arrived at washing station at: {env.now}")
      registration_time = 2
      print(f"{self.name} registering at washing station at: {env.now}")
      yield env.timeout(registration_time)
      try:
        with self.res.request() as req:
          yield req
          if(self.name == "Car_2"):
            self.env.process(interrupt(self.env,self))
          yield self.env.process(self.washing())
      except simpy.Interrupt:
        print(f"{self.name} was interrupted at {self.env.now}")
      yield self.env.process(self.leave())
      break
  def washing(self):
    print(f"The {self.name} is washing at {self.env.now}")
    yield env.timeout(10)
  def leave(self):
    print(f"The {self.name} is leaving the deck at {self.env.now}")
    yield env.timeout(1)

In [18]:
import simpy
env = simpy.Environment()
res = simpy.Resource(env, capacity=3)
for i in range(10):
  Car(env,res,f"Car_{i+1}")

In [19]:
env.run()

Car_1 arrived at washing station at: 0
Car_1 registering at washing station at: 0
Car_2 arrived at washing station at: 0
Car_2 registering at washing station at: 0
Car_3 arrived at washing station at: 0
Car_3 registering at washing station at: 0
Car_4 arrived at washing station at: 0
Car_4 registering at washing station at: 0
Car_5 arrived at washing station at: 0
Car_5 registering at washing station at: 0
Car_6 arrived at washing station at: 0
Car_6 registering at washing station at: 0
Car_7 arrived at washing station at: 0
Car_7 registering at washing station at: 0
Car_8 arrived at washing station at: 0
Car_8 registering at washing station at: 0
Car_9 arrived at washing station at: 0
Car_9 registering at washing station at: 0
Car_10 arrived at washing station at: 0
Car_10 registering at washing station at: 0
The Car_1 is washing at 2
The Car_2 is washing at 2
The Car_3 is washing at 2
Car_2 was interrupted at 4
The Car_2 is leaving the deck at 4
The Car_4 is washing at 4
The Car_1 is

# question 2

In [32]:
def interrupt(env,bill):
  yield env.timeout(0)
  bill.action.interrupt()
class biling:
  def __init__(self,env,res,name):
    self.env = env
    self.res = res
    self.name = name
    self.action = env.process(self.bill())
  def bill(self):
    unload_time = 2
    self.env.timeout(unload_time)
    try:
      with self.res.request() as req:
        yield req
        if self.name == "Customer_3":
          self.env.process(interrupt(self.env,self))
        yield self.env.process(self.start_biling())

    except simpy.Interrupt:
      print(f"{self.name} interupted at {self.env.now}")
    yield self.env.process(self.leaving())
  def start_biling(self):
    print(f"{self.name} biling started at {self.env.now}")
    yield self.env.timeout(10)
  def  leaving(self):
    print(f"{self.name} leaved at {self.env.now}")
    yield self.env.timeout(3)

In [33]:
import simpy
env = simpy.Environment()
res = simpy.Resource(env, capacity=4)
for i in range(10):
  biling(env,res,f"Customer_{i+1}")


In [34]:
env.run()

Customer_1 biling started at 0
Customer_2 biling started at 0
Customer_3 biling started at 0
Customer_4 biling started at 0
Customer_3 interupted at 0
Customer_3 leaved at 0
Customer_5 biling started at 0
Customer_1 leaved at 10
Customer_2 leaved at 10
Customer_4 leaved at 10
Customer_5 leaved at 10
Customer_6 biling started at 10
Customer_7 biling started at 10
Customer_8 biling started at 10
Customer_9 biling started at 10
Customer_6 leaved at 20
Customer_7 leaved at 20
Customer_8 leaved at 20
Customer_9 leaved at 20
Customer_10 biling started at 20
Customer_10 leaved at 30
